In [2]:
import pandas as pd
import os

In [10]:
repodir = os.getenv('REPO_DIR')
tripsdf = pd.read_csv(f'{repodir}/data/citibike/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{repodir}/learn-citibike/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [11]:
tripsdf.head()


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970325,504,1 Ave & E 15 St,40.732219,-73.981656,16950,Customer,\N,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002950,459,W 20 St & 11 Ave,40.746745,-74.007756,19816,Customer,\N,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,2
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718939,-73.992663,499,Broadway & W 60 St,40.769155,-73.981918,16063,Customer,\N,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,1


In [12]:
stationsdf.head()

,station_name,start station latitude,start station longitude,latlng,neighborhood,postal_code,state,sublocality
0,Bank St & Hudson St,40.736529,-74.006180,"40.73652889,-74.00618026",West Village,10014,NY,Manhattan
1,E 59 St & Sutton Pl,40.758491,-73.959206,"40.75849116,-73.95920622",Sutton Place,10022,NY,Manhattan
2,E 37 St & Lexington Ave,40.748238,-73.978311,"40.748238,-73.978311",Murray Hill,10016,NY,Manhattan
3,9 Ave & W 14 St,40.740583,-74.005509,"40.7405826,-74.00550867",Meatpacking District,10014,NY,Manhattan
4,Broadway & W 53 St,40.763441,-73.982681,"40.76344058,-73.98268129",Midtown West,10019,NY,Manhattan


In [13]:
tripsdf.head().iloc[0]

tripduration                               634
starttime                  2013-07-01 00:00:00
stoptime                   2013-07-01 00:10:34
start station id                           164
start station name             E 47 St & 2 Ave
start station latitude                 40.7532
start station longitude               -73.9703
end station id                             504
end station name               1 Ave & E 15 St
end station latitude                   40.7322
end station longitude                 -73.9817
bikeid                                   16950
usertype                              Customer
birth year                                  \N
gender                                       0
Name: 0, dtype: object

In [14]:
mdf = tripsdf[['start station name', 'end station name']
            ].merge(stationsdf[['station_name', 'neighborhood']], 
                    left_on='start station name',
                    right_on='station_name'
                   ).rename(columns={'neighborhood': 'start_neighborhood'}
                           ).merge(stationsdf[['station_name', 'neighborhood']],
                                  left_on='end station name',
                                   right_on='station_name'
                                  ).rename(columns={'neighborhood': 'end_neighborhood'})

In [15]:
tripsdf.shape, mdf.shape

((843416, 15), (843416, 6))

In [16]:
mdf.iloc[0]

start station name    E 47 St & 2 Ave
end station name      1 Ave & E 15 St
station_name_x        E 47 St & 2 Ave
start_neighborhood       Midtown East
station_name_y        1 Ave & E 15 St
end_neighborhood      Stuyvesant Town
Name: 0, dtype: object

In [17]:
statsdf = mdf[['start_neighborhood', 
               'end_neighborhood']].groupby(by=['start_neighborhood', 
               'end_neighborhood']).size().reset_index().rename(columns={0: 'count'})

In [18]:
statsdf['count'].sum()

843416

In [19]:
# What is the most popular destination overall?
mdf[['end_neighborhood']].groupby(by='end_neighborhood').size().reset_index(
            ).rename(columns={0: 'count'}).sort_values(by='count', ascending=False)

,end_neighborhood,count
8,Chelsea,92721
33,Midtown East,41296
22,Greenwich Village,40157
46,Tribeca,39614
23,Hell's Kitchen,37192
16,Financial District,36062
52,West Village,35993
34,Midtown West,30617
29,Lower East Side,26753
32,Midtown,26216


In [24]:
mdf['start_neighborhood'].unique().shape, mdf['end_neighborhood'].unique().shape

((54,), (54,))

In [29]:
from scipy.special import softmax
import numpy as np

In [37]:
x = np.array([[1, 0.5, 0.2, 3],
               [1,  -1,   7, 3],
               [2,  12,  13, 3]])
softmax(x, axis=1)[0].sum()
print('x[0]', x[0])
softmax(x[0])

x[0] [1.  0.5 0.2 3. ]


array([0.10587707, 0.06421769, 0.04757363, 0.78233161])

In [78]:

def make_probs(xdf):
    xdf['prob'] = softmax(xdf['count'])
    xdf['prob2'] = xdf['prob'].map(lambda x:round(x, ndigits=4))
    #return xdf[['end_neighborhood', 'prob']].T
    #blahdf = statsdf[statsdf['start_neighborhood'] == 'Alphabet City'].copy()
    #blahdf['prob'] = softmax(blahdf['count'])
    #statsdf.groupby(by='start_neighborhood').apply()
    return pd.DataFrame(dict(
        list(xdf[['end_neighborhood', 'prob2']].to_records(index=False))),
                       index=[0])




In [79]:
import datetime; import pytz
def utc_ts():
    return datetime.datetime.utcnow(
        ).replace(tzinfo=pytz.UTC).strftime('%Y-%m-%dT%H%M%SZ')
utc_ts()

'2020-06-06T034021Z'

In [81]:
hmmdf = statsdf.groupby(by='start_neighborhood').apply(make_probs)
work_dir = os.getenv('WORK_DIR')
statsdf.to_csv(f'{work_dir}/2020-05-31--citilearn/{utc_ts()}-statsdf.csv')
hmmdf.to_csv(f'{work_dir}/2020-05-31--citilearn/{utc_ts()}-hmmdf.csv')



In [82]:
hmmdf

,,Alphabet City,Battery Park City,Bedford-Stuyvesant,Boerum Hill,Bowery,Brooklyn Heights,Brooklyn Navy Yard,Central Park,Chelsea,Chinatown,...,Sutton Place,Theater District,Tribeca,Tudor City,Two Bridges,Ukrainian Village,Union Square,Vinegar Hill,West Village,Williamsburg
start_neighborhood,,,,,,,,,,,,,,,,,,,,,,
Alphabet City,0,1.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Battery Park City,0,0.0,1.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,NaN,0.0,0.0
Bedford-Stuyvesant,0,0.0,0.0,1.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Boerum Hill,0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,NaN,0.0000,0.0,...,0.0,0.0,0.000,NaN,0.0,0.0,0.0,0.0,0.0,0.0
Bowery,0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brooklyn Heights,0,0.0,0.0,0.0,0.0,0.0,1.0000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brooklyn Navy Yard,0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,NaN,0.0000,0.0,...,0.0,0.0,0.000,NaN,0.0,0.0,0.0,0.0,0.0,0.0
Central Park,0,0.0,0.0,0.0,NaN,0.0,0.0000,0.0,1.0,0.0000,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,NaN,0.0,0.0
Chelsea,0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,1.0000,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### So a super basic model that just predicts using the top prior probabity
- Let's use that to choose some metrics and try to evaluate a model
- So with this 54 neighborhood setup, we can say, a prediction on an input, 
is a vector of the probability of the 54 output neighborhoods. And we can make the evaluation
metric the multilabel logloss.


In [25]:

from sklearn.model_selection import train_test_split # (*arrays, **options)

In [ ]:
X, y 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [87]:
from sklearn.metrics import log_loss
import numpy as np
log_loss(["spam", "ham", "ham", "spam"],
         [[.1, .9], [.9, .1], [.8, .2], [.35, .65]])


0.21616187468057912

In [88]:
# Expecting near 0 for a perfect prediction
perfect_pred = np.array([[0, 0, 1],
                   [1, 0, 0],
                   [1, 0, 0],
                   [0, 0, 1],
                   [0, 1, 0]]) # [ham, jam, spam]
log_loss(["spam", "ham", "ham", "spam", "jam"],
         perfect_pred)


2.1094237467877998e-15

In [89]:
# ok cool and using numberic labels okay too?
perfect_pred = np.array([[0, 0, 1],
                   [1, 0, 0],
                   [1, 0, 0],
                   [0, 0, 1],
                   [0, 1, 0]]) # [ham, jam, spam]
log_loss([3, 1, 1, 3, 2],
         perfect_pred)


2.1094237467877998e-15

In [95]:
# and some noisy predictions? 
noisy_pred = np.array([[.05, .05, .9],
                   [.95, 0.05, 0],
                   [.9, 0.1, 0],
                   [0.05, .05, .9],
                   [0, 1, 0]]) # [ham, jam, spam]
log_loss([3, 1, 1, 3, 2],
         noisy_pred)


0.07347496827220674

In [164]:
# and some noisy predictions? 
noisy_pred = np.array([[.05, .05, .9, 0],
                   [.95, 0.05, 0, 0],
                   [.9, 0.1, 0, 0],
                   [0.05, .05, .9, 0],
                   [0, 1, 0, 0]]) # [ham, jam, spam]
log_loss([3, 1, 1, 3, 2],
         noisy_pred,
        labels=[1,2,3,4])


0.07347496827220787

#### Train this super dumb baseline model

In [96]:
np.array(["hi"])

array(['hi'], dtype='<U2')

#### Manual cross validation
- Doing this manually just because I had already written the train/fit using pandas
- And I want to just quickly do this

In [97]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape


((150, 4), (150,))

In [99]:

from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [100]:
pdb.runcall(cross_val_score, clf, X, y, cv=5)


> /usr/local/miniconda3/envs/pandars3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py(394)cross_val_score()
-> scorer = check_scoring(estimator, scoring=scoring)
(Pdb) l
389  	    :func:`sklearn.metrics.make_scorer`:
390  	        Make a scorer from a performance metric or loss function.
391  	
392  	    """
393  	    # To ensure multimetric format is not supported
394  ->	    scorer = check_scoring(estimator, scoring=scoring)
395  	
396  	    cv_results = cross_validate(estimator=estimator, X=X, y=y, groups=groups,
397  	                                scoring={'score': scorer}, cv=cv,
398  	                                return_train_score=False,
399  	                                n_jobs=n_jobs, verbose=verbose,
(Pdb) q


In [106]:
class FooFlassifiler():
    def __init__(self):
        return
    def fit(self, X, y):
        pass
    def score(self, X, y):
        return .9
    def get_params(self, deep):
        return {}
    
    

In [107]:
clf = FooFlassifiler()
scores = cross_val_score(clf, X, y, cv=5)


In [108]:
scores

array([0.9, 0.9, 0.9, 0.9, 0.9])

In [111]:
X, y = mdf[['start_neighborhood']].values, mdf['end_neighborhood'].tolist()

In [117]:
pd.DataFrame(X[:5], columns=['start_neighborhood']), y[:5]

(  start_neighborhood
 0       Midtown East
 1       Midtown East
 2       Midtown East
 3       Midtown East
 4       Midtown East,
 ['Stuyvesant Town',
  'Stuyvesant Town',
  'Stuyvesant Town',
  'Stuyvesant Town',
  'Stuyvesant Town'])

In [130]:
np.vstack((X[:5, 0], y[:5])).T
#X[:5,0]

array([['Midtown East', 'Stuyvesant Town'],
       ['Midtown East', 'Stuyvesant Town'],
       ['Midtown East', 'Stuyvesant Town'],
       ['Midtown East', 'Stuyvesant Town'],
       ['Midtown East', 'Stuyvesant Town']], dtype=object)

In [137]:
np.array(y[:5]).T

array(['Stuyvesant Town', 'Stuyvesant Town', 'Stuyvesant Town',
       'Stuyvesant Town', 'Stuyvesant Town'], dtype='<U15')

In [138]:
pd.DataFrame(np.vstack((X[:5, 0], y[:5])).T, columns=['start_neighborhood',
                                                   'end_neighborhood'])

,start_neighborhood,end_neighborhood
0,Midtown East,Stuyvesant Town
1,Midtown East,Stuyvesant Town
2,Midtown East,Stuyvesant Town
3,Midtown East,Stuyvesant Town
4,Midtown East,Stuyvesant Town


In [160]:
split_data = model.iloc[:5].to_dict(orient='split')
lookup = {a: np.array(split_data['data'])[i] for (i, a) in enumerate([x[0] for x in split_data['index']])}

In [232]:
len(split_data['columns'])

54

In [256]:
def dumb_fit(X, y):
    mdf = X_y_to_mydf(X, y)
    statsdf = mdf[['start_neighborhood', 
               'end_neighborhood']].groupby(by=['start_neighborhood', 
               'end_neighborhood']).size().reset_index().rename(columns={0: 'count'})
    hmmdf = statsdf.groupby(by='start_neighborhood').apply(make_probs).fillna(0)
    
    split_data = hmmdf.to_dict(orient='split')
    lookup_dict = {a: np.array(split_data['data'])[i] 
              for (i, a) in enumerate([x[0] for x in split_data['index']])}
    labels = split_data['columns']
    return hmmdf, lookup_dict, labels
    
def X_y_to_mydf(X, y):
    return pd.DataFrame(np.vstack((X[:, 0], y)).T, columns=['start_neighborhood',
                                                   'end_neighborhood'])

class SimpleFlassifiler():
    def __init__(self):
        return
    def fit(self, X, y):
        (self.lookup_df, self.lookup_dict, 
             self.labels) = dumb_fit(X, y)
        
    def score(self, X, y_true):
        y_preds = predict(self.lookup_dict, X)
        return log_loss(y_true, y_preds, labels=self.labels)

    def get_params(self, deep):
        return {}
    
def predict(lookup_dict, X):
    # return np.concatenate([lookup_dict[x[0]] for x in X])
    array_size = len(list(lookup_dict.values())[0])
    return np.concatenate([np.reshape(lookup_dict[x[0]], (1, array_size)) 
           for x in X])

In [249]:
len(list(lookup.values())[0])

54

In [238]:
# Make data from mdf
# (where mdf here just contains the ['start_neighborhood',
#                                    'end_neighborhood'] )
X, y = mdf[['start_neighborhood']].values, np.array(mdf['end_neighborhood'].tolist())
clf = SimpleFlassifiler()
clf.fit(X, y)
#model = dumb_fit(X, y)
#X.shape, y.shape
clf.score(X, y)

29.131042558171746

In [227]:
#clf.score(X, y)
#(X[0][0])
#'Midtown East' in 
#print(list(clf.lookup_dict.keys()))
# y_preds = predict(clf.lookup_dict, X)

In [235]:
clf.score(X, y)

29.131042558171746

In [213]:
#y_preds.shape, X.shape
array_size = len(clf.lookup_dict)
predarr = np.concatenate([np.reshape(clf.lookup_dict[x[0]], (1, array_size)) 
           for x in X])

In [242]:
X[:5], y[:5]

(array([['Midtown East'],
        ['Midtown East'],
        ['Midtown East'],
        ['Midtown East'],
        ['Midtown East']], dtype=object),
 array(['Stuyvesant Town', 'Stuyvesant Town', 'Stuyvesant Town',
        'Stuyvesant Town', 'Stuyvesant Town'], dtype='<U35'))

In [254]:
import ipdb

In [257]:

from sklearn.model_selection import cross_val_score
# clf = svm.SVC(kernel='linear', C=1)
clf = SimpleFlassifiler()
with ipdb.launch_ipdb_on_exception():
    scores = cross_val_score(clf, X, y, cv=5)


In [258]:
scores

array([29.03426394, 25.61716199, 29.19083979, 28.312853  , 22.04601817])

In [263]:
tripsdf.fillna('NaN').groupby(by=['usertype', 'gender']).size().reset_index()

,usertype,gender,0
0,Customer,0,174889
1,Customer,1,25
2,Customer,2,1
3,Subscriber,0,42
4,Subscriber,1,511454
5,Subscriber,2,157005
